In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm

### Import Precalculated Compustat Data

In [14]:
assets = pd.read_csv('/Users/olegbodrov/Hse/Thesis/Data/assets.csv', index_col='date')
factors = pd.read_csv('/Users/olegbodrov/Hse/Thesis/Data/factors.csv', index_col='date')
rm = pd.read_csv('/Users/olegbodrov/Hse/Thesis/Data/rm.csv', index_col='date')

In [315]:
for factor in factors.columns:
    print(factor)

AccrualsToAssets
BuybackYield
CashFlowToDebt
CashFlowYield
ClassicMomentumRelativeToBenchmark
ClassicMomentumRelativeToSP500
CommonEquityToAssets
CurrentPE
DebtToEquity
DepreciationToCapEx
DividendYield
EarningsYield
EBITDAToEnterpriseValue
FreeCashFlowYield
GrossMargin
LongTermDebtToAssets
OperatingIncomeYield
OperatingMargin
PriceToBook
PriceToCashFlow
PriceToSales
ROA
ROE
SalesYld
ShareholderYield
ClassicMomentum
AccrualsToTTMAssets
Debt
TradingVolumeUsd1M
SP500ClassicMomentum
MomentumResidual1M
MomentumResidual1Q
MomentumResidual1Yr
MomentumResidual6M
FFMse5Yr
FFRSquared5Yr
FFMse3Yr
FFRSquared3Yr
FFMse1Yr
FFRSquared1Yr
Beta5Yr
Beta3Yr
Beta1Yr
BenchmarkClassicMomentum
DebtGrowth1Yr
DebtGrowth5Yr
DivYieldChange1Yr
DivYieldChange5Yr
EarningsChange1YrToCap
EarningsChange1YrToSales
EarningsChange5YrToCap
EarningsChange5YrToSales
EpsGrowth1Yr
EpsGrowth5Yr
FCFChange1YrToCap
FCFChange1YrToSales
FCFChange5YrToCap
FCFChange5YrToSales
FCFYldChange1Yr
FCFYldChange5Yr
FreeCashFlowGrowth1Yr
Free

### Evaluate contribution of test_factors

### Data preparation

In [34]:
test_factors_name = [
    'MomentumResidual1M',
    'MomentumResidual6M',
    'MomentumResidual1Q',
    'MomentumResidual1Yr',
    'MomentumResidualClassic'
]

In [227]:
factors = factors.apply(lambda x: (x - x.mean()) / x.std())

In [231]:
# Substract benchmark return from assets
assets = assets - (np.array(rm).reshape((len(rm), 1)) * np.ones((1, len(assets.columns))))

In [232]:
# Choose control factors
control_factors_name = [e for e in factors.columns if e not in test_factors]

In [233]:
# Divide factors' data in control and test groups
control_factors = factors[control_factors_name]
test_factors = factors[test_factors_name]

In [234]:
# Calculate matrix of covariances of assets' returns with control factors
Ch = pd.DataFrame(
    index=assets.columns,
    columns=control_factors.columns
)
for i in assets.columns:
    for j in control_factors.columns:
        index = assets[assets[i].notnull()].loc[:, i].index.intersection(
            factors[factors[j].notnull()].loc[:, j].index
        )
        Ch.loc[i, j] = np.cov(
            np.array(assets.loc[index, i]),
            np.array(control_factors.loc[index, j])
        )[0, 1]

<ipython-input-234-e2fdce9177ea>:11: RuntimeWarning: Degrees of freedom <= 0 for slice
  Ch.loc[i, j] = np.cov(
/Users/olegbodrov/Library/Python/3.8/lib/python/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Users/olegbodrov/Library/Python/3.8/lib/python/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [316]:
Ch.head()

,AccrualsToAssets,BuybackYield,CashFlowToDebt,CashFlowYield,ClassicMomentumRelativeToBenchmark,ClassicMomentumRelativeToSP500,CommonEquityToAssets,CurrentPE,DebtToEquity,DepreciationToCapEx,...,EPSChage2YrToPrice,EPSChage3YrToPrice,IdiosyncraticRiskMSE,IdiosyncraticRiskR2,Alpha1Yr,Alpha3Yr,Alpha5Yr,DividendGrowth1Yr,DividendGrowth3Yr,DividendGrowth5Yr
00100401,-0.010284,0.005102,-0.024166,0.009499,-0.002938,0.000205,0.000061,-0.0078,0.00308,0.028375,...,-0.015544,-0.016324,-0.00441,0.006216,-0.001745,-0.014038,-0.017354,-0.025633,-0.026653,-0.02508
00104504,0.005521,0.014173,-0.012615,0.007677,-0.009676,-0.007407,-0.009532,-0.01183,0.001867,0.005074,...,0.00002,-0.003845,-0.010092,-0.02118,-0.011877,-0.010438,-0.008626,-0.008079,-0.014115,-0.00939
00107501,-0.004838,-0.015967,-0.001036,0.014711,0.022798,0.022757,-0.023221,-0.002651,0.024537,-0.004805,...,0.009111,0.013483,-0.062161,0.039889,0.029906,0.023715,0.028553,0.021566,0.016957,0.012822
00107601,-0.000792,-0.003071,-0.003138,0.012879,0.013648,0.013648,-0.009707,-0.008208,0.013849,-0.005065,...,0.006254,0.010681,-0.024908,0.034424,0.016128,0.006878,0.00763,0.002959,0.004004,0.00455
00107801,0.003084,-0.00781,0.017101,0.000186,0.028087,0.029601,-0.015702,0.008084,0.012421,-0.008857,...,0.014895,0.020471,-0.054731,0.034264,0.034454,0.029993,0.034626,0.033531,0.03501,0.032078


In [235]:
# Calculate matrix of covariances of assets' returns with test factors
Cg = pd.DataFrame(
    index=assets.columns,
    columns=test_factors.columns
)
for i in assets.columns:
    for j in test_factors.columns:
        index = assets[assets[i].notnull()].loc[:, i].index.intersection(
            factors[factors[j].notnull()].loc[:, j].index
        )
        Cg.loc[i, j] = np.cov(
            np.array(assets.loc[index, i]),
            np.array(test_factors.loc[index, j])
        )[0, 1]

<ipython-input-235-deceb1cbd1d0>:11: RuntimeWarning: Degrees of freedom <= 0 for slice
  Cg.loc[i, j] = np.cov(


In [317]:
Cg.head()

,MomentumResidual1M,MomentumResidual6M,MomentumResidual1Q,MomentumResidual1Yr,MomentumResidualClassic
00100401,-0.002743,0.010886,0.00607,0.006768,0.007939
00104504,-0.017077,-0.005515,0.001383,-0.016767,-0.007737
00107501,-0.001893,0.022268,0.014811,0.028007,0.025049
00107601,0.004485,0.014272,0.009222,0.017764,0.015799
00107801,-0.00026,0.025887,0.016798,0.033761,0.028874


In [236]:
# Calculate average return for each asset across time
# It will be our regressand
r_avg = assets.mean()

In [237]:
# Check for NaNs
Ch[Ch.isna().any(axis=1)]

,AccrualsToAssets,BuybackYield,CashFlowToDebt,CashFlowYield,ClassicMomentumRelativeToBenchmark,ClassicMomentumRelativeToSP500,CommonEquityToAssets,CurrentPE,DebtToEquity,DepreciationToCapEx,...,EPSChage2YrToPrice,EPSChage3YrToPrice,IdiosyncraticRiskMSE,IdiosyncraticRiskR2,Alpha1Yr,Alpha3Yr,Alpha5Yr,DividendGrowth1Yr,DividendGrowth3Yr,DividendGrowth5Yr
03738201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03744501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
Cg[Cg.isna().any(axis=1)]

,MomentumResidual1M,MomentumResidual6M,MomentumResidual1Q,MomentumResidual1Yr,MomentumResidualClassic
03738201,NaN,NaN,NaN,NaN,NaN
03744501,NaN,NaN,NaN,NaN,NaN


In [239]:
# Unfortunately we should say goodbuy to two observations
Ch.drop(['03738201', '03744501'], axis=0, inplace=True)
Cg.drop(['03738201', '03744501'], axis=0, inplace=True)
r_avg.drop(['03738201', '03744501'], axis=0, inplace=True)

### 1st Lasso. Get the parsimonius factor model in explaining cross-section returns

In [297]:
lasso_1 = linear_model.LassoCV(cv=10, n_alphas=200, max_iter=100000)
lasso_1.fit(Ch, r_avg)
lasso_1_coef = lasso_1.coef_

In [298]:
lasso_1.alpha_

3.5912447809481654e-07

In [299]:
# Select factors with non zero coefficients
factors_to_ols = []
for i, e in enumerate(lasso_1_coef):
    if e != 0:
        factor = control_factors.columns[i]
        factors_to_ols.append(factor)
        print(factor)

AccrualsToAssets
BuybackYield
CashFlowYield
ClassicMomentumRelativeToSP500
CommonEquityToAssets
DebtToEquity
DepreciationToCapEx
DividendYield
EarningsYield
EBITDAToEnterpriseValue
FreeCashFlowYield
GrossMargin
LongTermDebtToAssets
OperatingIncomeYield
OperatingMargin
PriceToBook
PriceToCashFlow
ROA
ROE
ShareholderYield
ClassicMomentum
Debt
SP500ClassicMomentum
FFMse5Yr
FFRSquared5Yr
FFRSquared3Yr
FFMse1Yr
Beta1Yr
BenchmarkClassicMomentum
DebtGrowth1Yr
DebtGrowth5Yr
DivYieldChange1Yr
DivYieldChange5Yr
EarningsChange1YrToCap
EarningsChange1YrToSales
EpsGrowth1Yr
EpsGrowth5Yr
FCFChange1YrToCap
FCFChange1YrToSales
FCFYldChange1Yr
FCFYldChange5Yr
FreeCashFlowGrowth1Yr
FreeCashFlowGrowth5Yr
NetOperatingAssetsGrowth1Yr
NetOperatingAssetsGrowth5Yr
OperatingMarginGrowth1Yr
OperatingMarginGrowth5Yr
SalesGrowth1Yr
SalesGrowth5Yr
SalesPerShareGrowth1Yr
SalesYldGrowth1Yr
SalesYldGrowth5Yr
SalesPerShareGrowth3Yr
EPSChage2YrToPrice
IdiosyncraticRiskR2
Alpha1Yr
Alpha3Yr
DividendGrowth1Yr
DividendGrow

In [113]:
# Factors with zero lasso coef
for i, e in enumerate(lasso_1_coef):
    if e == 0:
        factor = control_factors.columns[i]
        print(factor)

CashFlowToDebt
ClassicMomentumRelativeToBenchmark
CurrentPE
PriceToSales
ROA
SalesYld
AccrualsToTTMAssets
FFMse3Yr
FFRSquared1Yr
Beta5Yr
BenchmarkClassicMomentum
EarningsChange5YrToSales
FCFChange5YrToCap
FCFChange5YrToSales
FCFYldChange1Yr
NetOperatingAssetsGrowth5Yr
OperatingEpsGrowth1Yr
SalesPerShareGrowth5Yr
SalesPerShareGrowth2Yr
EPSChage1YrToPrice
EPSChage2YrToPrice
EPSChage3YrToPrice
IdiosyncraticRiskMSE
Alpha3Yr


### Alternative to CV. Manually choose penalty in Lasso.

In [322]:
lasso_1.alpha_

3.5912447809481654e-07

In [323]:
lasso_1_alt = linear_model.Lasso(alpha=0.000001, max_iter=100000)
lasso_1_alt.fit(Ch, r_avg)
lasso_1_alt_coef = lasso_1_alt.coef_

In [324]:
# Select factors with non zero coefficients
factors_to_ols_alt = []
for i, e in enumerate(lasso_coef):
    if e != 0:
        factor = control_factors.columns[i]
        factors_to_ols_alt.append(factor)
        print(factor)

EBITDAToEnterpriseValue
FFMse1Yr
SalesYldGrowth1Yr
SalesPerShareGrowth3Yr
EPSChage3YrToPrice
IdiosyncraticRiskR2
Alpha3Yr
DividendGrowth1Yr


In [325]:
new_factor = 'MomentumResidualClassic'

In [326]:
lasso_2_alt = linear_model.Lasso(alpha=0.0001, max_iter=100000)
lasso_2_alt.fit(Ch, Cg[new_factor])
lasso_2_alt.coef_

array([ 0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.74908307,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.01812786,  0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        ,  0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.  

In [327]:
lasso_2_alt_coef = lasso_2_alt.coef_

In [328]:
# We will take only second regressor because it remains non-zero with more than one penalty value
for i, e in enumerate(lasso_2_alt_coef):
    if e != 0:
        factor = control_factors.columns[i]
        if factor not in factors_to_ols_alt:
            factors_to_ols_alt.append(factor)
            print(factor)

ClassicMomentumRelativeToSP500
ClassicMomentum


### Estimate SDF coefficients of chosen factors by OLS regression

In [308]:
# Post-selection estimation (OLS)
X = pd.concat([Cg[new_factor], Ch[factors_to_ols_alt]], axis=1, join='inner')
ols = linear_model.LinearRegression()
ols.fit(X, r_avg)
lambda_ = ols.coef_

In [309]:
ols.intercept_

0.00028428837115797875

In [276]:
lambda_

array([-0.77732136,  0.05888569,  0.17753288,  0.35776146, -0.02644425,
        0.32713985,  0.05227839,  0.68078323, -0.60538215, -0.00205923,
        0.50840049,  0.50840049])

In [277]:
for i, e in enumerate([new_factor] + factors_to_ols_alt):
    print(e, ' ', lambda_[i])

MomentumResidualClassic   -0.7773213635500208
EBITDAToEnterpriseValue   0.05888568626869686
FFMse1Yr   0.1775328753382895
SalesYldGrowth1Yr   0.35776146021258404
SalesPerShareGrowth3Yr   -0.026444254121689492
EPSChage3YrToPrice   0.32713984844948274
IdiosyncraticRiskR2   0.052278387376487476
Alpha3Yr   0.6807832300994796
DividendGrowth1Yr   -0.605382151474844
BuybackYield   -0.0020592348947901136
ClassicMomentumRelativeToSP500   0.5084004858111467
ClassicMomentum   0.5084004858111472


### Test inference of new factor SDF loading (OLS coefficient)

In [278]:
# Select factors for SDF
sdf_factors = []
sdf_factors.append(new_factor)
for f in factors_to_ols_alt:
    sdf_factors.append(f)
sdf_factors = factors[sdf_factors]

In [279]:
i_factors = []
for i, e in enumerate(lasso_2_coef):
    if e != 0:
        factor = control_factors.columns[i]
        i_factors.append(factor)

In [280]:
# Calculate teta and z by OLS regression of new factor on I
ols_inf = linear_model.LinearRegression()
ols_inf.fit(control_factors[i_factors], test_factors[new_factor])

LinearRegression()

In [281]:
# teta
ols_inf.coef_

array([-0.03495907,  0.02981405,  0.47388619,  0.11290891,  0.09428343,
        0.02286703, -0.07188326,  0.47388619, -0.00697097,  0.01782316,
        0.01980922, -0.01352134, -0.05651995, -0.11709531,  0.0964091 ,
        0.06474397, -0.03784935,  0.40395406, -0.13245134,  0.0081192 ])

In [282]:
# z
z = test_factors[new_factor] - ols_inf.predict(control_factors[i_factors])

In [283]:
# sigma z
sigma_z = pow(z, 2).sum() / len(test_factors)

In [284]:
lambda_ = np.array(lambda_).T

In [285]:
# П without NW 
T = len(lambda_)
summ = 0
for i in range(T):
    sdf_factors_t = np.array(sdf_factors.iloc[i, :])
    summ += pow((1 - np.matmul(lambda_, sdf_factors_t)), 2) * pow(z[i], 2)/(pow(sigma_z, 2))
summ = summ / T
print(summ)

85.88276841585362


In [286]:
# Newy-West (param = 12)
q = 12
summ_nw = 0
for k in range(q):
    for t in range(k+1, T+1):
        sdf_factors_t = np.array(sdf_factors.iloc[t, :])
        sdf_factors_t_k = np.array(sdf_factors.iloc[t-k, :])
        summ_nw += (1 - k+1/(q+1+1))*(1 - np.matmul(lambda_, sdf_factors_t))*(
            1 - np.matmul(lambda_, sdf_factors_t_k))*2*(z[t]*z[t-k])/(pow(sigma_z, 2))
summ_nw = summ_nw / t
print(summ_nw)    

50.730111503745


In [287]:
summ = (summ + summ_nw) / T
summ

11.384406659966551

In [288]:
np.sqrt(summ / T)

0.9740126051531431

In [310]:
# Critical value of statistic
-0.7773213635500208 / 0.9740126051531431

-0.7980608869305169